### Year 1 Topology - Limit Checking - Overloading, Voltage limits

In [1]:
import pandas as pd

##### Loading

##### Percentage Loading in Branches 

Calculation related can be found in PSSE PAGVI Manual Chapter 6 Power system network simulations - Section 6.6.7 Activity Rate 

%I = MVA(actual)/(MVA(rated)*Vpu)

Vpu -> Bus Voltage at from end 

In [2]:
list_all = []
list_gens = [0,50,100]
list_lsc = ['lls','rls','hls']
for gen in list_gens:
    for lsc in list_lsc:
        file_csv_out = 'savnw_sol_' + str(gen) +'_'+ lsc + '_loading.csv'
        data_load = pd.read_csv(file_csv_out,header=1, skiprows=0)
        # Extracted data contains branch loading from the metered and non-metered ends. The data is filtered to obtain the branch loading from the metered end.
        data_fil = data_load[data_load['From Bus']!=data_load['Non Metered Bus']].reset_index(drop=True)
        data_fil['From Bus V'] = data_fil['From Bus Ext'].str.extract(r'(\d+\.\d+)') 
        data_fil['To Bus V'] = data_fil['To Bus Ext'].str.extract(r'(\d+\.\d+)')
        data_fil['Scenario']= 'Solar = ' + str(gen) + ' MW, ' + lsc.upper()
        list_all.append(data_fil)
data_all = pd.concat(list_all).reset_index(drop=True)

##### Checking transformer loading greater than 80% 

In [3]:
transformer_loading = data_all[data_all['From Bus V']!=data_all['To Bus V']].reset_index(drop=True)
transformer_loading[transformer_loading['%MVA'] > 80.0]

,From Bus,From Bus Ext,To Bus,To Bus Ext,Ckt ID,Non Metered Bus,Branch I,Branch Flow MW,Branch Flow MVAR,Branch MVA,MW Loss,MVAR Loss,Rate,%I,%MVA,From Bus V,To Bus V,Scenario


##### Checking branch loading greater than 80% 

In [4]:
branch_loading = data_all[data_all['From Bus V'] == data_all['To Bus V']].reset_index(drop=True)
branch_loading[branch_loading['%I'] > 80.0]

,From Bus,From Bus Ext,To Bus,To Bus Ext,Ckt ID,Non Metered Bus,Branch I,Branch Flow MW,Branch Flow MVAR,Branch MVA,MW Loss,MVAR Loss,Rate,%I,%MVA,From Bus V,To Bus V,Scenario


##### Voltage Upper Limits 

In [5]:
list_volt = []
list_gens = [0,50,100]
list_lsc = ['lls','rls','hls']
for gen in list_gens:
    for lsc in list_lsc:
        file_csv_out = 'savnw_sol_' + str(gen) +'_'+ lsc + '_volt.csv'
        data_volt = pd.read_csv(file_csv_out,header=1, skiprows=0)
        data_volt['Scenario']= 'Solar = ' + str(gen) + ' MW, ' + lsc.upper()
        list_volt.append(data_volt)
data_all_volt = pd.concat(list_volt).reset_index(drop=True)
data_all_volt = data_all_volt[['Scenario','Bus Number', 'Bus Name', 'Area', 'Base Voltage', 'Bus Voltage(PU)','Bus Voltage(kV)']]

In [6]:
data_all_volt[data_all_volt['Bus Voltage(PU)']>1.05].reset_index(drop=True)

,Scenario,Bus Number,Bus Name,Area,Base Voltage,Bus Voltage(PU),Bus Voltage(kV)
0,"Solar = 0 MW, LLS",201,HYDRO,2,500.0,1.059,529.269
1,"Solar = 0 MW, LLS",3018,CATDOG_G,5,13.8,1.058,14.605
2,"Solar = 0 MW, RLS",201,HYDRO,2,500.0,1.056,528.058
3,"Solar = 0 MW, RLS",3018,CATDOG_G,5,13.8,1.055,14.559
4,"Solar = 0 MW, HLS",201,HYDRO,2,500.0,1.054,527.084
5,"Solar = 0 MW, HLS",3018,CATDOG_G,5,13.8,1.052,14.523
6,"Solar = 50 MW, LLS",201,HYDRO,2,500.0,1.060,529.935
7,"Solar = 50 MW, LLS",3018,CATDOG_G,5,13.8,1.059,14.612
8,"Solar = 50 MW, RLS",201,HYDRO,2,500.0,1.058,528.765
9,"Solar = 50 MW, RLS",3018,CATDOG_G,5,13.8,1.056,14.567


##### Voltage Lower Limits

In [7]:
data_all_volt[data_all_volt['Bus Voltage(PU)']<0.95]

,Scenario,Bus Number,Bus Name,Area,Base Voltage,Bus Voltage(PU),Bus Voltage(kV)


#### Conclusion 

Year 1 Toplogy limit violation check for overloading and voltage was carried out for the three load scenarios. It was seen that
1. no overloading was reported for any branches including transformer branches for considered solar output and load scenarios 
2. The buses violating the upper voltage limit for various scenarios are:
    * Solar = 0 MW, Scenarios = LLS, RLS, HLS
      BUS 201, 308
    * Solar = 50 MW, Scenarios = LLS, RLS, HLS
      BUS 201, 308
    * Solar = 100 MW, Scenarios = LLS
      BUS 152, 201, 308
    * Solar = 100 MW, Scenarios = RLS, HLS
      BUS 201, 308